In [2]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails

In [3]:
azureml.core.VERSION

'1.0.39'

In [4]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

In [6]:
ws = Workspace.from_config()

In [8]:
def_file_storage = ws.get_default_datastore()

In [9]:
def_blob_storage = Datastore(ws, "workspaceblobstore")

In [10]:
def_blob_storage.name

'workspaceblobstore'

In [11]:
project_folder = '.'

In [12]:
def_file_storage.upload_files(['./20news.pkl'], target_path='20newsgroups', overwrite=False, show_progress=True)

Uploading ./20news.pkl
Uploaded ./20news.pkl, 1 files out of an estimated total of 1


$AZUREML_DATAREFERENCE_1e0e997ee41f411396ab0f6ec15714a0

In [14]:
def_blob_storage.upload_files(['./20news.pkl'], target_path='20newsgroups', overwrite=True, show_progress=True)

Uploading ./20news.pkl
Uploaded ./20news.pkl, 1 files out of an estimated total of 1


$AZUREML_DATAREFERENCE_acbc4ea29909465c9353a2a741bab6b7

In [16]:
aml_compute = ws.compute_targets['prodmod-compute1']

In [17]:
step1 = PythonScriptStep(name='train_step', script_name='train.py', compute_target=aml_compute, 
                         source_directory=project_folder, allow_reuse=True)

In [18]:
step2 = PythonScriptStep(name="compare_step",
                         script_name="compare.py", 
                         compute_target=aml_compute, 
                         source_directory=project_folder)

In [19]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

In [20]:
run_config = RunConfiguration()

In [21]:
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config.environment.python.user_managed_dependencies = False
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [22]:
step3 = PythonScriptStep(name='extract_step', script_name='extract.py', compute_target=aml_compute, 
                         source_directory=project_folder, runconfig=run_config)

In [23]:
steps=[step1, step2, step3]

In [24]:
pipeline1 = Pipeline(workspace=ws, steps=steps)

In [25]:
pipeline1.validate()

Step train_step is ready to be created [903c6dea]
Step compare_step is ready to be created [fde33afc]
Step extract_step is ready to be created [076623f4]


[]

In [26]:
pipeline_run1 = Experiment(ws, 'Hello_world1').submit(pipeline1, regenerate_outputs=False)

Created step train_step [903c6dea][a8b253ce-6c13-4d12-887d-63607f3f3478], (This step will run and generate new outputs)
Created step compare_step [fde33afc][673bc594-b167-4690-8439-cee8873de0c5], (This step will run and generate new outputs)
Created step extract_step [076623f4][01f70a66-1c4b-4384-bac9-5671e4450333], (This step will run and generate new outputs)
Submitted pipeline run: 8e993a5c-69e3-47a0-82ff-0ac1fe984001


In [27]:
RunDetails(pipeline_run1).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [29]:
step_runs = pipeline_run1.get_children()
for step_run in step_runs:
    status = step_run.get_status()
    print('Script:', step_run.name, 'status:', status)
    
    # Change this if you want to see details even if the Step has succeeded.
    if status == "Failed":
        joblog = step_run.get_job_log()
        print('job log:', joblog)

Script: train_step status: Finished
Script: compare_step status: Finished
Script: extract_step status: Running


In [30]:
step2.run_after(step1)
step3.run_after(step2)

# Try a loop
#step2.run_after(step3)

# Now, construct the pipeline using the steps.

# We can specify the "final step" in the chain, 
# Pipeline will take care of "transitive closure" and 
# figure out the implicit or explicit dependencies
# https://www.geeksforgeeks.org/transitive-closure-of-a-graph/
pipeline2 = Pipeline(workspace=ws, steps=[step3])
print ("Pipeline is built")

pipeline2.validate()
print("Simple validation complete")

pipeline_run2 = Experiment(ws, 'Hello_World2').submit(pipeline2)
print("Pipeline is submitted for execution")

RunDetails(pipeline_run2).show()

Pipeline is built
Step extract_step is ready to be created [778e5074]
Step compare_step is ready to be created [91aa1729]
Simple validation complete
Created step extract_step [778e5074][da92f381-bf98-4bf8-a11c-46fbd27604a5], (This step will run and generate new outputs)
Created step compare_step [91aa1729][ef9d8b1e-f7d2-4f32-9764-c73c4f7db313], (This step will run and generate new outputs)
Created step train_step [c3ec194d][a8b253ce-6c13-4d12-887d-63607f3f3478], (This step is eligible to reuse a previous run's output)
Submitted pipeline run: fa8864af-e963-4ae1-b505-75f7f2d118fa
Pipeline is submitted for execution


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …